## Animating the network-area-diagram: bringing the grid to life!
  - This notebook demonstrates how to **visualize network states over time**.
  - We will build a time-indexed dataset and drive a diagram with a time slider.

#### 1. Load the network snapshots
- We load about 30 snapshots of the french network, starting two hours before the 7pm peak hour and ending 15 minutes after.
- We build a timestamp dataframe based on
   - p1, p2 values
   - connected1, connected2 values
- For efficiency purpose, biidm is used instead of xiidm/jiidm

In [1]:
import pypowsybl.network as pn
import pandas as pd

def filepath(ts: str) -> str:
    return base_dir + '/fr_' + ts + '.biidm'

base_dir = '/home/dupuyflo/Data/LFE_2025'
timestamps = ['1645', '1650', '1655',
              '1700', '1705', '1710', '1715', '1720', '1725', '1730', '1735', '1740', '1745', '1750', '1755',
              '1800', '1805', '1810', '1815', '1820', '1825', '1830', '1835', '1840', '1845', '1850', '1855',
              '1900', '1905', '1910', '1915']

rows = []
for ts in timestamps:
    path = filepath(ts)
    network = pn.load(path)
    branches = network.get_branches(attributes=["p1", "p2", "connected1", "connected2"])
    branches = branches.reset_index()
    timestamp_df = pd.DataFrame({
        'branch_id': branches['id'],
        'timestamp': pd.to_datetime(ts, format="%H%M"),
        'value1': branches['p1'].fillna(0.0),
        'value2': branches['p2'].fillna(0.0),
        'connected1': branches['connected1'],
        'connected2': branches['connected2']
    })
    rows.append(timestamp_df)

time_series_df = pd.concat(rows, ignore_index=True)
time_series_df

,branch_id,timestamp,value1,value2,connected1,connected2
0,.CTLHL31.CTLO,1900-01-01 16:45:00,0.000000,0.000000,True,False
1,.CTLHL32.CTLO,1900-01-01 16:45:00,0.000000,0.000000,True,True
2,.CTLOL31FINS,1900-01-01 16:45:00,9.227890,-9.181527,True,True
3,.CTLOL31ZLIEB,1900-01-01 16:45:00,-9.227890,9.311742,True,True
4,.G.ROL51HOSPI,1900-01-01 16:45:00,0.000000,0.000023,False,True
...,...,...,...,...,...,...
291581,WARANY763,1900-01-01 19:15:00,-247.733219,247.928738,True,True
291582,WARANY764,1900-01-01 19:15:00,-240.771758,240.952718,True,True
291583,WEPPEY761,1900-01-01 19:15:00,-208.978923,209.114225,True,True
291584,WEPPEY762,1900-01-01 19:15:00,-210.034189,210.165402,True,True


- searching the branches whose connection state changes during the studied time slot

In [2]:
changing_c1 = time_series_df.groupby('branch_id')['connected1'].nunique() > 1
changing_c2 = time_series_df.groupby('branch_id')['connected2'].nunique() > 1
changing_branches = (changing_c1 | changing_c2)
changing_branches[changing_branches].index.tolist()

['ANOULL31SSDIE',
 'ARNAGY641',
 'ARNAGY643',
 'BAIXAL32I.TET',
 'BENODL31CONCA',
 'BREN5L61VERFE',
 'CHEV6L41CPFL5',
 'CONCAL61SQUIV',
 'COTONL42R.LES',
 'COUDOL31SIGNE',
 'CROZEL31ZCAP5',
 'EPER5L41RAMBO',
 'ESCO2L31ESCOU',
 'FOYAUL42P.SAM',
 'LAFIGL61ZLAFI',
 'LANG5L31ZKERS',
 'MAGN5L41SOUTE',
 'MARL6L61MOLSH',
 'MOLSHY631',
 'MTIGNL41ZC.LA',
 'P.GASY763',
 'P.GASY765',
 'PRRTTL61VERTE',
 'QUIMPL31ZKERA',
 'SSCROY611']

#### 2. Displaying the results
- Again, we use the pseudo-geographical fixed nad positions
- Two limitations:
    - the time slider is only available in nad_explorer widget so far
    - only active power values are updated so far 

- Reading the pseudo-geographical nad positions 

In [3]:
metadata = pd.read_json('/home/dupuyflo/Documents/LFE_2025/france_metadata.json')
fixed_positions = pd.DataFrame({
    "id": metadata["equipmentId"],
    "x": metadata["x"],
    "y": metadata["y"]
})
fixed_positions.set_index('id', drop=True, inplace=True)

- Displaying the nad_explorer widget

In [4]:
from pypowsybl_jupyter import nad_explorer

nad_explorer(network,voltage_level_ids= ['CONCAP6'], depth=3, time_series_data=time_series_df, fixed_nad_positions=fixed_positions)